In [ ]:
import torch
import cv2
import numpy as np
import pytesseract
from datetime import datetime
from deep_sort_realtime.deepsort_tracker import DeepSort
from collections import defaultdict

In [ ]:
# Load YOLOv5 model (pre-trained on COCO dataset)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Initialize Deep SORT tracker
tracker = DeepSort(max_age=30, n_init=3, nn_budget=100)

# Path to Tesseract executable (for Windows users)
pytesseract.pytesseract.tesseract_cmd = r'Tesseract-OCR\tesseract.exe'


In [ ]:
def extract_license_plate(frame, bbox):
    """Extracts the license plate region from the vehicle bounding box."""
    x1, y1, x2, y2 = bbox
    plate_img = frame[y1:y2, x1:x2]  # Crop the license plate area
    gray = cv2.cvtColor(plate_img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)  # Binarize the image
    return thresh


In [ ]:
def recognize_plate(plate_img):
    """Uses Tesseract OCR to recognize text from the plate image."""
    text = pytesseract.image_to_string(plate_img, config='--psm 8')
    return ''.join(char for char in text if char.isalnum()).upper()

In [ ]:
vehicle_data = defaultdict(lambda: {'plate': None, 'entry_time': None, 'exit_time': None, 'toll_fee': 0})

In [ ]:
def calculate_distance(coord1, coord2):
    """Calculates Euclidean distance between two points."""
    return np.linalg.norm(np.array(coord1) - np.array(coord2))

In [ ]:
def compute_toll_fee(distance):
    """Computes toll fee based on distance traveled."""
    return 0.5 * distance  # Example: 0.5 currency units per distance unit

In [ ]:
def track_vehicle_movement(frame, detections):
    """Tracks vehicle movement and assigns unique IDs."""
    formatted_detections = [(bbox[:4], bbox[4], int(bbox[5])) for bbox in detections]
    tracks = tracker.update_tracks(formatted_detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed() or track.time_since_update > 1:
            continue

        track_id = track.track_id
        bbox = track.to_ltwh()
        x, y, w, h = bbox

        if vehicle_data[track_id]['plate'] is None:
            plate_img = extract_license_plate(frame, (x, y, x + w, y + h))
            plate_text = recognize_plate(plate_img)
            vehicle_data[track_id]['plate'] = plate_text
            vehicle_data[track_id]['entry_time'] = datetime.now()

        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, f"ID: {track_id} | Plate: {vehicle_data[track_id]['plate']}",
                    (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # Simulate toll point detection
        entry_point, exit_point = (0, 0), (500, 500)
        distance = calculate_distance(entry_point, exit_point)
        vehicle_data[track_id]['toll_fee'] = compute_toll_fee(distance)
        vehicle_data[track_id]['exit_time'] = datetime.now()


In [ ]:
def main(video_source):
    cap = cv2.VideoCapture(video_source)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)
        detections = results.xyxy[0].cpu().numpy()
        vehicle_detections = [d for d in detections if int(d[5]) in [2, 3, 5, 7]]
        frame = track_vehicle_movement(frame, vehicle_detections)

        cv2.imshow("Vehicle Tracking", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

main(0)
